# Top-10 RAWG per Award Category
This notebook computes a deduplicated Top-10 list (by rating) for each award category you provided, using only the RAWG data file `data/RAWG/rawg_data.csv`.
- It deduplicates games by name (keeps the highest rating).
- It computes a comparable `rating_score` preferring RAWG `metacritic` when available, otherwise using `ratings` scaled to 0–100.
- Matching to a category is done via vectorized case-insensitive substring matching across `genres`, `tags`, and `name` where present.
Outputs: per-category CSVs written to `data/processed/top10_rawg_<category_slug>.csv` and a combined `data/processed/top10_rawg_all.csv`.

In [1]:
# Imports and display settings
import os, re
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 80)
print('pandas', pd.__version__)

pandas 2.3.3


In [2]:
# Load RAWG cleaned data (single source for this notebook)
import os
rawg_path_candidates = [
    'data/RAWG/rawg_data.csv',
    'data/rawg/rawg_data.csv',
    'data/processed/rawg_cleaned.csv',
    '../data/RAWG/rawg_data.csv',
    '../data/rawg/rawg_data.csv',
]
rawg_path = None
for p in rawg_path_candidates:
    if os.path.exists(p):
        rawg_path = p
        break
if rawg_path is None:
    raise FileNotFoundError(f'RAWG data not found. Tried: {rawg_path_candidates}')
rawg = pd.read_csv(rawg_path, dtype=str).fillna('')
print('loaded', rawg.shape, 'columns:', list(rawg.columns), 'from', rawg_path)
rawg.head(3)

loaded (2360, 11) columns: ['rawg_id', 'rawg_slug', 'name', 'release_date', 'genres', 'tags', 'ratings', 'platforms', 'esrb', 'metacritic', 'description'] from ../data/RAWG/rawg_data.csv


,rawg_id,rawg_slug,name,release_date,genres,tags,ratings,platforms,esrb,metacritic,description
0,303576,vampire-the-masquerade-bloodlines-2,Vampire: The Masquerade – Bloodlines 2,2025-10-21,Action|RPG,Singleplayer|Для одного игрока|Экшен|Приключен...,3.85,PC|PlayStation 5|Xbox One|PlayStation 4|Xbox S...,,,"Sired in an act of vampire insurrection, your ..."
1,58386,stalker-2,S.T.A.L.K.E.R. 2: Heart of Chornobyl,2024-11-20,Shooter|Adventure|Action|RPG,Singleplayer|Для одного игрока|Экшен|Steam Ach...,3.77,PC|Xbox Series S/X,,,S.T.A.L.K.E.R. 2 is a brand-new entry in the l...
2,292844,hollow-knight-silksong,Hollow Knight: Silksong,2025-09-04,Indie|Platformer|Adventure|Action,Singleplayer|Для одного игрока|Экшен|Приключен...,4.33,PC|PlayStation 5|Xbox One|PlayStation 4|Xbox S...,,,Hollow Knight: Silksong is the epic sequel to ...


In [3]:
# Preprocess: ensure name column, numeric rating columns, and compute a vectorized rating_score
def ensure_name_col(df):
    if 'name' in df.columns:
        return df
    for alt in ['game_name','title']:
        if alt in df.columns:
            df['name'] = df[alt].astype(str)
            return df
    df['name'] = ''
    return df
rawg = ensure_name_col(rawg)
# numeric conversions
if 'metacritic' in rawg.columns:
    rawg['metacritic'] = pd.to_numeric(rawg['metacritic'], errors='coerce')
if 'ratings' in rawg.columns:
    rawg['ratings'] = pd.to_numeric(rawg['ratings'], errors='coerce')
# compute rating_score vectorized: prefer metacritic, else ratings*20
rawg['rating_score'] = np.nan
if 'metacritic' in rawg.columns:
    rawg.loc[rawg['metacritic'].notna(), 'rating_score'] = rawg.loc[rawg['metacritic'].notna(), 'metacritic'].astype(float)
if 'ratings' in rawg.columns:
    mask = rawg['rating_score'].isna() & rawg['ratings'].notna()
    rawg.loc[mask, 'rating_score'] = rawg.loc[mask, 'ratings'].astype(float) * 20.0
# normalize name_key for deduplication
rawg['name_key'] = rawg['name'].astype(str).str.strip().str.lower()
# deduplicate: keep highest rating_score per name_key (tie-breaker: metacritic, ratings)
rawg_sorted = rawg.sort_values(['rating_score', 'metacritic', 'ratings'], ascending=[False, False, False], na_position='last')
rawg_dedup = rawg_sorted.drop_duplicates(subset=['name_key'], keep='first').reset_index(drop=True)
print('deduplicated rows:', rawg_dedup.shape)
rawg_dedup.head(3)

deduplicated rows: (2360, 13)


,rawg_id,rawg_slug,name,release_date,genres,tags,ratings,platforms,esrb,metacritic,description,rating_score,name_key
0,337273,dispatch-2,Dispatch,2025-10-22,Adventure|Action|Casual|Strategy|Indie,Atmospheric|Horror|Pixel Graphics|Dark|creepy|...,4.77,PC|PlayStation 5,Adults Only,NaN,Dispatch is a superhero workplace comedy where...,95.4,dispatch
1,998492,system-shock-2-25th-anniversary-remaster,System Shock 2: 25th Anniversary Remaster,2025-06-25,Adventure|Action|RPG,Singleplayer|Для одного игрока|Экшен|Приключен...,4.62,PC,Mature,NaN,Accolades\nAbout the GameSystem Shock® 2: 25th...,92.4,system shock 2: 25th anniversary remaster
2,974379,dragon-ball-sparking-zero,DRAGON BALL: Sparking! ZERO,2025-10-10,Action|Fighting,Singleplayer|Для одного игрока|Экшен|Steam Ach...,4.60,PC|PlayStation 5|Xbox Series S/X|Nintendo Switch,Teen,NaN,DRAGON BALL: Sparking! ZERO takes the legendar...,92.0,dragon ball: sparking! zero


In [4]:
# Setup category keywords, search fields and output directory
outdir = 'data/processed'
os.makedirs(outdir, exist_ok=True)
results = []
# pick fields available in the RAWG dataframe to search over
search_fields = [f for f in ['genres','tags','name','description','short_description'] if f in rawg_dedup.columns]
if not search_fields:
    # fallback to 'name' so at least titles are matched
    search_fields = ['name']
# Simple keyword lists for common award categories.
# Edit these if you have more precise category keyword lists.
CATEGORY_KEYWORDS = {
    'Best Narrative': ['story','narrative','plot','character','dialogue'],
    'Best Art Direction': ['art','visual','graphics','art direction','pixel art','visuals'],
    'Best Audio': ['sound','music','audio','score','soundtrack'],
    'Best Indie': ['indie','independent'],
    'Best Multiplayer': ['multiplayer','co-op','coop','online','pvp'],
    'Best RPG': ['rpg','role-playing','role playing','jrpg','crpg'],
    'Best Shooter': ['shooter','fps','gunplay','shooting'],
    'Best Strategy': ['strategy','turn-based','tbs','4x','rts','real-time'],
    'Best Sports/Racing': ['sports','racing','football','soccer','basketball','motorsport'],
    # Added categories requested by user:
    'Best Action': ['action','fast-paced','beat-em-up','beat em up','hack and slash','action-adventure'],
    'Best Adventure': ['adventure','exploration','open world','point-and-click','point and click','adventure game'],
    'Best Fighting': ['fighting','fighter','brawler','versus','vs','one-on-one','beat em up'],
    'Best Family': ['family','kids','family-friendly','children','party game']
}


In [5]:
# For each category, build a regex from the keywords and apply vectorized .str.contains across search fields
import re

def make_regex(keywords):
    # escape and join, match word substrings (case-insensitive)
    kws = [re.escape(k) for k in keywords if k.strip()]
    if not kws:
        return None
    return '(' + '|'.join(kws) + ')'

top10_by_cat = {}
for cat, kws in CATEGORY_KEYWORDS.items():
    regex = make_regex(kws)
    if not regex:
        top10_by_cat[cat] = pd.DataFrame()
        continue
    # boolean masks across fields
    masks = []
    for f in search_fields:
        masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
    full_mask = pd.Series(False, index=rawg_dedup.index)
    if masks:
        full_mask = masks[0]
        for m in masks[1:]:
            full_mask = full_mask | m
    sel = rawg_dedup[full_mask].copy()
    if sel.empty:
        top10_by_cat[cat] = pd.DataFrame()
        print(cat, '-> no matches')
        continue
    sel = sel.dropna(subset=['rating_score']).sort_values('rating_score', ascending=False).head(10)
    top10_by_cat[cat] = sel
    # safe slug
    slug = re.sub(r'[^a-z0-9_]+','', cat.replace(' ','_').lower())
    rpath = os.path.join(outdir, f'top10_rawg_{slug}.csv')
    sel.to_csv(rpath, index=False)
    print('Wrote', rpath, '->', len(sel), 'rows')
    # collect for combined table
    tmp = sel.copy()
    tmp['category'] = cat.replace('Best ','')
    tmp['source'] = 'RAWG'
    results.append(tmp)

# combined CSV
if results:
    combined = pd.concat(results, ignore_index=True)
    combined.to_csv(os.path.join(outdir, 'top10_rawg_all.csv'), index=False)
    print('Wrote combined top10_rawg_all.csv with', len(combined), 'rows')
else:
    print('No top10 rows were produced')

/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_narrative.csv -> 10 rows


Wrote data/processed/top10_rawg_best_art_direction.csv -> 10 rows


/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_audio.csv -> 10 rows
Wrote data/processed/top10_rawg_best_indie.csv -> 10 rows


...

Wrote data/processed/top10_rawg_best_adventure.csv -> 10 rows


/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_fighting.csv -> 10 rows


/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/3906681518.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_family.csv -> 10 rows
Wrote combined top10_rawg_all.csv with 130 rows


In [6]:
# For each category, build a regex from the keywords and apply vectorized .str.contains across search fields
import re

def make_regex(keywords):
    # escape and join, match word substrings (case-insensitive)
    kws = [re.escape(k) for k in keywords if k.strip()]
    if not kws:
        return None
    return '(' + '|'.join(kws) + ')'

top10_by_cat = {}
for cat, kws in CATEGORY_KEYWORDS.items():
    regex = make_regex(kws)
    if not regex:
        top10_by_cat[cat] = pd.DataFrame()
        continue
    # boolean masks across fields
    masks = []
    for f in search_fields:
        masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
    full_mask = pd.Series(False, index=rawg_dedup.index)
    if masks:
        full_mask = masks[0]
        for m in masks[1:]:
            full_mask = full_mask | m
    sel = rawg_dedup[full_mask].copy()
    if sel.empty:
        top10_by_cat[cat] = pd.DataFrame()
        print(cat, '-> no matches')
        continue
    sel = sel.dropna(subset=['rating_score']).sort_values('rating_score', ascending=False).head(10)
    top10_by_cat[cat] = sel
    slug = re.sub(r'[^a-z0-9_]+','', cat.replace(' ','_').lower())
    rpath = os.path.join(outdir, f'top10_rawg_{slug}.csv')
    sel.to_csv(rpath, index=False)
    print('Wrote', rpath, '->', len(sel), 'rows')
    # collect for combined table
    tmp = sel.copy()
    tmp['category'] = cat.replace('Best ','')
    tmp['source'] = 'RAWG'
    results.append(tmp)

# combined CSV
if results:
    combined = pd.concat(results, ignore_index=True)
    combined.to_csv(os.path.join(outdir, 'top10_rawg_all.csv'), index=False)
    print('Wrote combined top10_rawg_all.csv with', len(combined), 'rows')
else:
    print('No top10 rows were produced')

/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_narrative.csv -> 10 rows


/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_art_direction.csv -> 10 rows


/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

...

Wrote data/processed/top10_rawg_best_adventure.csv -> 10 rows


/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_fighting.csv -> 10 rows


/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex, case=False, na=False))
/tmp/ipykernel_17447/2569439283.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  masks.append(rawg_dedup[f].astype(str).str.contains(regex,

Wrote data/processed/top10_rawg_best_family.csv -> 10 rows
Wrote combined top10_rawg_all.csv with 260 rows


In [7]:
# Top 10 — Best Narrative
cat = 'Best Narrative'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Narrative
                                           Game  Rating
      System Shock 2: 25th Anniversary Remaster    92.4
                    DRAGON BALL: Sparking! ZERO    92.0
                    Clair Obscur: Expedition 33    90.4
                                  Split Fiction    88.8
One Iced Latte With Your Breast Milk, Please! ☕    88.6
                                       Hades II    88.0
                    The Rise of the Golden Idol    87.6
                   Kingdom Come: Deliverance II    87.4
                                     The Alters    87.0
                        Hollow Knight: Silksong    86.6


In [8]:
# Top 10 — Best Art Direction
cat = 'Best Art Direction'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Art Direction
                                           Game  Rating
                                       Dispatch    95.4
                    DRAGON BALL: Sparking! ZERO    92.0
     Legacy of Kain™ Soul Reaver 1&2 Remastered    91.4
                    Clair Obscur: Expedition 33    90.4
                                  Split Fiction    88.8
One Iced Latte With Your Breast Milk, Please! ☕    88.6
                    The Rise of the Golden Idol    87.6
               Amerzone - The Explorer's Legacy    86.6
             Indiana Jones and the Great Circle    86.4
                                     Ball x Pit    86.4


In [9]:
# Top 10 — Best Audio
cat = 'Best Audio'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Audio
                   Game  Rating
             The Alters    87.0
Hollow Knight: Silksong    86.6
            The Drifter    86.0
       Sword of the Sea    85.0
                 MiSide    83.8
   Monster Hunter Wilds    83.6
         Tempest Rising    83.6
          Battlefield 6    82.8
Ninja Gaiden: Ragebound    82.2
               Megabonk    81.0


In [10]:
# Top 10 — Best Indie
cat = 'Best Indie'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Indie
                                           Game  Rating
                                       Dispatch    95.4
                                     Bionic Bay    88.6
One Iced Latte With Your Breast Milk, Please! ☕    88.6
                                       Hades II    88.0
                    The Rise of the Golden Idol    87.6
                        Hollow Knight: Silksong    86.6
                                     Ball x Pit    86.4
                                    The Drifter    86.0
                              The Midnight Walk    86.0
                              Monument Valley 3    85.8


In [11]:
# Top 10 — Best Multiplayer
cat = 'Best Multiplayer'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Multiplayer
                                     Game  Rating
System Shock 2: 25th Anniversary Remaster    92.4
              DRAGON BALL: Sparking! ZERO    92.0
                            Split Fiction    88.8
                     Monster Hunter Wilds    83.6
                           Tempest Rising    83.6
                            RV There Yet?    83.4
                            Battlefield 6    82.8
                              Blue Prince    82.6
                                    PEAK.    81.6
                           Infinity Nikki    81.6


In [12]:
# Top 10 — Best RPG
cat = 'Best RPG'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best RPG
                                     Game  Rating
System Shock 2: 25th Anniversary Remaster    92.4
              Clair Obscur: Expedition 33    90.4
                                 Hades II    88.0
             Kingdom Come: Deliverance II    87.4
                               Ball x Pit    86.4
                                 Cabernet    85.8
                                   MiSide    83.8
                     Monster Hunter Wilds    83.6
                      Lies of P: Overture    83.6
                           Infinity Nikki    81.6


In [13]:
# Top 10 — Best Shooter
cat = 'Best Shooter'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Shooter
                                     Game  Rating
System Shock 2: 25th Anniversary Remaster    92.4
       Indiana Jones and the Great Circle    86.4
                      Doom: The Dark Ages    86.4
                               Ball x Pit    86.4
                            Battlefield 6    82.8
                                 Megabonk    81.0
                           Beyond Citadel    80.0
                     Cronos: The New Dawn    78.6
                          Slime Rancher 2    78.4
                   Dying Light: The Beast    78.4


In [14]:
# Top 10 — Best Strategy
cat = 'Best Strategy'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Strategy
                       Game  Rating
                   Dispatch    95.4
DRAGON BALL: Sparking! ZERO    92.0
Clair Obscur: Expedition 33    90.4
                 Ball x Pit    86.4
          Monument Valley 3    85.8
       The King is Watching    84.0
                     MiSide    83.8
                     Awaria    83.6
             Tempest Rising    83.6
        Strange Antiquities    83.4


In [15]:
# Top 10 — Best Sports/Racing
cat = 'Best Sports/Racing'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Sports/Racing
                         Game  Rating
                   Bionic Bay    88.6
             Mario Kart World    82.4
                    Despelote    80.0
Lonely Mountains: Snow Riders    80.0
                    CloverPit    77.8
                  Wheel World    77.2
       Cash Cleaner Simulator    77.2
                        CRUEL    71.4
                 The Precinct    67.8
                    SUPERVIVE    66.6


In [16]:
# Top 10 — Best Action
cat = 'Best Action'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('Top 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)

# Top 10 — Best Adventure
cat = 'Best Adventure'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('\nTop 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)

# Top 10 — Best Fighting
cat = 'Best Fighting'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('\nTop 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)

# Top 10 — Best Family
cat = 'Best Family'
if cat in top10_by_cat and not top10_by_cat[cat].empty:
    df = top10_by_cat[cat][['name','rating_score']].copy()
    df = df.rename(columns={'name':'Game','rating_score':'Rating'}).reset_index(drop=True)
    print('\nTop 10 —', cat)
    print(df.to_string(index=False))
else:
    print('No matches for', cat)


Top 10 — Best Action
                                      Game  Rating
                                  Dispatch    95.4
 System Shock 2: 25th Anniversary Remaster    92.4
               DRAGON BALL: Sparking! ZERO    92.0
Legacy of Kain™ Soul Reaver 1&2 Remastered    91.4
               Clair Obscur: Expedition 33    90.4
                             Split Fiction    88.8
                                Bionic Bay    88.6
                                  Hades II    88.0
           Death Stranding 2: On The Beach    87.4
              Kingdom Come: Deliverance II    87.4

Top 10 — Best Adventure
                                           Game  Rating
                                       Dispatch    95.4
      System Shock 2: 25th Anniversary Remaster    92.4
                    DRAGON BALL: Sparking! ZERO    92.0
     Legacy of Kain™ Soul Reaver 1&2 Remastered    91.4
                    Clair Obscur: Expedition 33    90.4
                                  Split Fiction    88.8
 

In [17]:
# Top 10 — Overall Best Games
# Compute the top 10 games overall by rating_score from the deduplicated RAWG table
best_overall = rawg_dedup.dropna(subset=['rating_score']).sort_values('rating_score', ascending=False).head(10).copy()
# select some useful columns if available
cols = ['name','rating_score']
for c in ['platforms','release_date','genres']:
    if c in best_overall.columns:
        cols.append(c)
cols = [c for c in cols if c in best_overall.columns]
best_overall = best_overall[cols]
# rename for display
best_overall = best_overall.rename(columns={'name':'Game','rating_score':'Rating'})
print('Top 10 — Overall Best Games')
print(best_overall.reset_index(drop=True).to_string(index=False))
# write CSV
os.makedirs(outdir, exist_ok=True)
best_overall.to_csv(os.path.join(outdir, 'top10_rawg_best_overall.csv'), index=False)
print('\nWrote', os.path.join(outdir, 'top10_rawg_best_overall.csv'))


Top 10 — Overall Best Games
                                           Game  Rating                                                                   platforms release_date                                          genres
                                       Dispatch    95.4                                                            PC|PlayStation 5   2025-10-22          Adventure|Action|Casual|Strategy|Indie
      System Shock 2: 25th Anniversary Remaster    92.4                                                                          PC   2025-06-25                            Adventure|Action|RPG
                    DRAGON BALL: Sparking! ZERO    92.0                            PC|PlayStation 5|Xbox Series S/X|Nintendo Switch   2025-10-10                                 Action|Fighting
     Legacy of Kain™ Soul Reaver 1&2 Remastered    91.4     PC|PlayStation 5|Xbox One|PlayStation 4|Xbox Series S/X|Nintendo Switch   2024-12-10              Platformer|Adventure|Action|Puzzle
       

## Notes
- Matching is heuristic (substring matching). If you want stricter or broader matching, we can refine the keyword lists or use fuzzy matching.
- Deduplication is done by case-normalized `name`. If you want to merge similar names (editions, DLC), I can add fuzzy grouping.
- Files written to `data/processed/` for easy consumption by dashboards.